In [1]:
import time
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from datetime import datetime
from selenium.common.exceptions import UnexpectedAlertPresentException
import os
import glob
import shutil
import logging
import pandas as pd

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
# 작업경로 초기화
def init_folder(download_path):
    if os.path.exists(download_path):
    # 폴더가 존재하면 내용을 모두 삭제
        for filename in os.listdir(download_path):
            file_path = os.path.join(download_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                logging.warning(f'Failed to delete {file_path}. Reason: {e}')
    else:
        # 폴더가 존재하지 않으면 새로 생성
        os.makedirs(download_path)

    logging.info("'downloads' 폴더가 정리되었습니다.")

# 다운로드 폴더와 원하는 파일 이름 설정
def downloadCsv(download_url, download_path):

    input_fieldcode = "TRDSTATENM" #(영업상태명) 필드코드
    input_text = "영업"

    # Chrome WebDriver 설정
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "profile.default_content_settings.popups": 0
    })

    # WebDriver 초기화
    driver = webdriver.Chrome(options=options)

    # 웹페이지 열기
    driver.get(download_url)
    driver.maximize_window()

    try:
        select_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "selt_filterCol"))
    )
        select = Select(select_element)
        select.select_by_value(input_fieldcode)
        get_fieldcode = select_element.get_attribute('value')
        if get_fieldcode == input_fieldcode:
            logging.info(f"{input_fieldcode} 값이 정상적으로 입력되었습니다.")
        else:
            logging.warning(f"값이 올바르게 입력되지 않았습니다. 예상값: {input_fieldcode}, 실제값: {get_fieldcode}")        

        text_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "txtFilter"))
        )
        text_element.clear()
        text_element.send_keys(input_text)
        get_text = text_element.get_attribute('value')
        if get_text == input_text:
            logging.info(f"{input_text} 값이 정상적으로 입력되었습니다.")
        else:
            logging.warning(f"값이 올바르게 입력되지 않았습니다. 예상값: {input_text}, 실제값: {get_text}")

        # '조회' 버튼 찾기 및 클릭
        inquire_button = driver.find_element(By.XPATH, "//button[@class='btn-base-s'][.//span[text()='조회']]")
        inquire_button.click()
    
        time.sleep(3)

        # 'CSV 내려받기' 버튼 찾기 및 클릭
        csv_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "btnCsv"))
        )
        csv_button.click()

        # 알림 창 처리
        try:
            WebDriverWait(driver, 5).until(EC.alert_is_present())
            alert = driver.switch_to.alert
            logging.info(f"알림 메시지: {alert.text}")
            alert.accept()
        except:
            logging.warning("알림 창이 나타나지 않았습니다.")

        logging.info("CSV 파일 다운로드가 시작되었습니다.")

        csv_files = glob.glob(os.path.join(download_path, "*.csv"))
        if csv_files:
            logging.info(f"CSV 파일 다운로드가 완료되었습니다: {csv_files[0]}")
        else:
            logging,Warning("CSV 파일 다운로드가 실패했거나 아직 완료되지 않았습니다.")

    # 다운로드 완료 대기
        time.sleep(3)  # 필요에 따라 시간 조정

    except UnexpectedAlertPresentException as e:
        logging.warning(f"예상치 못한 알림 창 발생: {e.alert_text}")
        driver.switch_to.alert.accept()

    except Exception as e:
        logging.warning(f"오류 발생: {e}")

    finally:
        # 브라우저 종료
        driver.quit()
    return(csv_files[0])

# csv파일을 읽어서 df생성
def read_csv(csv_name):
    df = pd.DataFrame()
    encodings = ['utf-8', 'cp949', 'euc-kr']
    for encoding in encodings:
        try:
            df = pd.read_csv(csv_name, 
                             usecols=['관리번호', '전화번호', '지번주소', '도로명주소', '사업장명', '업태구분명'],
                             dtype={'관리번호': str, '전화번호': str, '지번주소': str, '도로명주소': str, '사업장명': str, '업태구분명': str},
                             encoding=encoding)
            logging.info(f"파일을 {encoding} 인코딩으로 성공적으로 읽었습니다.")
            print(df.head())  # 데이터 확인
            return df
        except Exception as e:
            logging.warning(f"{encoding} 인코딩으로 읽기 실패: {e}")
    
    logging.warning("모든 인코딩 시도 실패")
    return None

now = datetime.now()
download_url = "https://data.seoul.go.kr/dataList/OA-18665/S/1/datasetView.do"
download_path = os.path.join(os.getcwd(), "downloads") #현재 작업 디렉토리(os.getcwd())

init_folder(download_path)
csv_name = downloadCsv(download_url,download_path)
#csv_name = glob.glob(os.path.join(download_path, "*.csv"))[0]
csv_data = read_csv(csv_name)
if csv_data is not None:
    logging.info(f"총 {len(csv_data)} 개의 행이 로드되었습니다.")
else:
    logging.warning("CSV 파일 읽기에 실패했습니다.")

2024-09-09 08:58:15,425 - INFO - 'downloads' 폴더가 정리되었습니다.
2024-09-09 08:58:27,408 - WARNING - Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
2024-09-09 08:58:44,446 - INFO - TRDSTATENM 값이 정상적으로 입력되었습니다.
2024-09-09 08:58:44,645 - INFO - 영업 값이 정상적으로 입력되었습니다.
2024-09-09 08:58:53,503 - WARNING - 알림 창이 나타나지 않았습니다.
2024-09-09 08:58:53,504 - INFO - CSV 파일 다운로드가 시작되었습니다.
2024-09-09 08:58:53,505 - INFO - CSV 파일 다운로드가 완료되었습니다: c:\Start\python_basic\KakaoMap\downloads\서울시 마포구 일반음식점 인허가 정보.csv
2024-09-09 08:59:02,784 - WARNING - utf-8 인코딩으로 읽기 실패: 'utf-8' codec can't decode byte 0xb0 in position 0: invalid start byte
2024-09-09 08:59:02,858 - INFO - 파일을 cp949 인코딩으로 성공적으로 읽었습니다.
2024-09-09 08:59:02,867 - INFO - 총 8235 개의 행이 로드되었습니다.


                     관리번호        전화번호                            지번주소  \
0  3130000-101-1970-02282  02 7162413  서울특별시 마포구 용강동 494-61번지 2층, 3층    
1  3130000-101-1971-00749  0207178906           서울특별시 마포구 용강동 40-1번지    
2  3130000-101-1971-00754  0207166661          서울특별시 마포구 용강동 50-14번지    
3  3130000-101-1971-08714  02 7179021            서울특별시 마포구 도화동 179-7    
4  3130000-101-1973-00744  02 7027462             서울특별시 마포구 용강동 43-2    

                            도로명주소    사업장명  업태구분명  
0  서울특별시 마포구 토정로 268 (용강동,2층, 3층)     부영각    중국식  
1         서울특별시 마포구 토정로 313 (용강동)     태순집     한식  
2     서울특별시 마포구 토정로 312 (용강동, 2층)  마포옥하우스     한식  
3        서울특별시 마포구 새창로2길 13 (도화동)    도화곱창     한식  
4         서울특별시 마포구 토정로 311 (용강동)    조박집투  호프/통닭  


In [3]:
now.strftime("%y%m%d")

'240909'